In [1]:
import numpy as np
import tensorflow as tf


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [24]:

# get data from train_1.csv
tmp = np.genfromtxt("train_2.csv", dtype=float, delimiter=",")

# set training_data quantity and validation data quantity
train_quantity = 700
validation_quantity = 891-train_quantity

# 归一化
tem = tmp[1:,3:]
ave = np.mean(tem,axis = 0)*np.ones((891,7))
ptp = np.ptp(tem,axis = 0)*np.ones((891,7))
tem = np.subtract(tem,ave)
tem = np.true_divide(tem,ptp)
tmp[1:,3:] = tem

#apart train,validation
train_data = tmp[1:train_quantity+1,3:]
validation_data0 = tmp[train_quantity+1:,3:]

train_lable = tmp[1:train_quantity+1,2]
validation_lable0 = tmp[train_quantity+1:,2]

train_lable = train_lable.reshape(train_quantity,1)
validation_lable0 = validation_lable0.reshape(validation_quantity,1)

# define layer function
def add_layer(input,in_size,out_size):
    theta = tf.Variable(tf.random_normal([in_size,out_size]))
    if out_size==1:
        z = tf.matmul(input,theta)
        a = tf.nn.relu(z)
    else:
        z = tf.matmul(input,theta)
        a = tf.sigmoid(z)
        a_0 = tf.ones([train_quantity,1])
        a = tf.concat([a_0,a],1)
    return theta,a,z


# define placeholder for inputs to network
xs = tf.placeholder(tf.float32,[None,7])
ys = tf.placeholder(tf.float32,[None,1])

# add hidden layer
theta_1,l1,z_1 = add_layer(xs,7,8)
# add output layer
theta_2,output,z_2 = add_layer(l1,9,1)

#regularzation
lamda = 0.055
regularzation = (tf.reduce_sum(tf.square(theta_1))+tf.reduce_sum(tf.square(theta_2)))*lamda/600

# the error between prediction and real data
J_theta = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels = ys,logits = z_2)) + regularzation

a =  3
# learning rate
train_step = tf.train.GradientDescentOptimizer(a).minimize(J_theta)

#prediction
validation_data = tf.constant(validation_data0,tf.float32)
validation_lable = tf.constant(validation_lable0,tf.float32)

a_1 = tf.nn.relu(tf.matmul(validation_data,theta_1))
a_0 = tf.ones([validation_quantity,1])
a_20 = tf.concat([a_0,a_1],1)
a_2 = tf.sigmoid(tf.matmul(a_20,theta_2))
prediction = tf.round(a_2)

validation = tf.subtract(prediction,validation_lable)
validation = tf.abs(validation)
accuracy =tf.reduce_sum(validation) 
accuracy = (validation_quantity-accuracy)/validation_quantity

#initialize
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

for i in range(10000):
    #training
    sess.run(train_step,feed_dict={xs:train_data,ys:train_lable})
    
    if i%500 == 0:
        loss = sess.run(J_theta,feed_dict={xs:train_data,ys:train_lable})
        print("J_theta:",loss)
        acc = sess.run(accuracy)
        print(acc)
        




J_theta: 0.9401188
0.5497382
J_theta: 0.9401188
0.5497382
J_theta: 0.4547772
0.82722515
J_theta: 0.4547772
0.82722515
J_theta: 0.45296392
0.8167539
J_theta: 0.45296392
0.8167539
J_theta: 0.45097074
0.8115183
J_theta: 0.45097074
0.8115183
J_theta: 0.44908062
0.8115183
J_theta: 0.44908062
0.8115183
J_theta: 0.4479257
0.8167539
J_theta: 0.4479257
0.8167539
J_theta: 0.44729355
0.8167539
J_theta: 0.44729355
0.8167539
J_theta: 0.44700244
0.8115183
J_theta: 0.44700244
0.8115183
J_theta: 0.44688004
0.82198954
J_theta: 0.44688004
0.82198954
J_theta: 0.44682556
0.82198954
J_theta: 0.44682556
0.82198954
J_theta: 0.4467982
0.82198954
J_theta: 0.4467982
0.82198954
J_theta: 0.446782
0.82198954
J_theta: 0.446782
0.82198954
J_theta: 0.44677013
0.82198954
J_theta: 0.44677013
0.82198954
J_theta: 0.44675937
0.82198954
J_theta: 0.44675937
0.82198954
J_theta: 0.44674787
0.8167539
J_theta: 0.44674787
0.8167539
J_theta: 0.44673443
0.8167539
J_theta: 0.44673443
0.8167539
J_theta: 0.44671857
0.8115183
J_theta:

In [12]:
# import test sets
test = np.genfromtxt("test_2.csv",dtype = float,delimiter=",")

#reshape test sets
tem1 = test[1:,2:]
ave1 = np.mean(tem1,axis = 0)*np.ones((418,7))
ptp1 = np.ptp(tem1,axis = 0)*np.ones((418,7))
tem1 = np.subtract(tem1,ave1)
tem1 = np.true_divide(tem1,ptp1)
test[1:,2:] = tem1
test = test[1:,2:]

# define tensorflow constant test data
test_data = tf.constant(test,tf.float32)

# predict test data
t_1 = tf.nn.relu(tf.matmul(test_data,theta_1))
t_0 = tf.ones([418,1])
t_20 = tf.concat([t_0,t_1],1)
t_2 = tf.sigmoid(tf.matmul(t_20,theta_2))
prediction_1 = tf.round(t_2)
prediction_1 = tf.to_int32(prediction_1)

# return result
result = sess.run(prediction_1)
No = np.arange(892,1310)
No = No.reshape(418,1)
title = np.array(["PassengerId","Survived"])
title = title.reshape(1,2)
result = np.append(No,result,axis=1)
result = np.append(title,result,axis=0)
np.savetxt("result.csv", result, delimiter=",",fmt = "%s")


In [8]:
sess.run(t_2)

array([[0.08189662],
       [0.3430517 ],
       [0.09308192],
       [0.10264447],
       [0.42918906],
       [0.223799  ],
       [0.65176725],
       [0.1467574 ],
       [0.8471118 ],
       [0.07613342],
       [0.08622137],
       [0.27874768],
       [0.95579493],
       [0.04016088],
       [0.89447695],
       [0.94234985],
       [0.21987648],
       [0.23679549],
       [0.44098356],
       [0.7283265 ],
       [0.3257728 ],
       [0.31505603],
       [0.95380205],
       [0.47410884],
       [0.96530247],
       [0.02761948],
       [0.97248375],
       [0.21865408],
       [0.30837223],
       [0.07910178],
       [0.05819552],
       [0.11706539],
       [0.34227893],
       [0.35423088],
       [0.41413984],
       [0.27053386],
       [0.48502657],
       [0.53124446],
       [0.11598787],
       [0.1156124 ],
       [0.09765748],
       [0.3880305 ],
       [0.0500249 ],
       [0.85523576],
       [0.89905614],
       [0.11483041],
       [0.41038546],
       [0.102

array([[0.08189662],
       [0.3430517 ],
       [0.09308192],
       [0.10264447],
       [0.42918906],
       [0.223799  ],
       [0.65176725],
       [0.1467574 ],
       [0.8471118 ],
       [0.07613342],
       [0.08622137],
       [0.27874768],
       [0.95579493],
       [0.04016088],
       [0.89447695],
       [0.94234985],
       [0.21987648],
       [0.23679549],
       [0.44098356],
       [0.7283265 ],
       [0.3257728 ],
       [0.31505603],
       [0.95380205],
       [0.47410884],
       [0.96530247],
       [0.02761948],
       [0.97248375],
       [0.21865408],
       [0.30837223],
       [0.07910178],
       [0.05819552],
       [0.11706539],
       [0.34227893],
       [0.35423088],
       [0.41413984],
       [0.27053386],
       [0.48502657],
       [0.53124446],
       [0.11598787],
       [0.1156124 ],
       [0.09765748],
       [0.3880305 ],
       [0.0500249 ],
       [0.85523576],
       [0.89905614],
       [0.11483041],
       [0.41038546],
       [0.102